In [ ]:
import pandas as pd
import numpy as np
import re
import findspark
findspark.init('/home/dave/spark-2.4.1-bin-hadoop2.7/')
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import *
import sys
import os

datafile = "/home/dave/Documents/CS691/proj/data/snotel/SNOTEL_518_Heavenly_Valley_CA"

spark = SparkSession.builder.appName("proj_exp").getOrCreate()

df = spark.read.text(datafile)

df= df.filter(~ df.value.rlike('#'))
df= df.filter(~ df.value.rlike('Date'))

#split DF into Cols: Date,Station Name,Station Id,Snow Water Equivalent (in),Change In Snow Water Equivalent (in),
#Snow Depth (in),Change In Snow Depth (in),Air Temperature Observed (degF),Barometric Pressure (inch_Hg),Dew Point 
#Temperature (degF),Wind Direction Observed (degree),Wind Speed Observed (mph)
split_col = pyspark.sql.functions.split(df['value'], ',')

df = df.withColumn('Date', split_col.getItem(0))
df = df.withColumn('Station Name', split_col.getItem(1))
df = df.withColumn('sid', split_col.getItem(2))
df = df.withColumn('swe', split_col.getItem(3))
df = df.withColumn('dswe', split_col.getItem(4))
df = df.withColumn('sd', split_col.getItem(5))
df = df.withColumn('dsd', split_col.getItem(6))
df = df.withColumn('at', split_col.getItem(7))

w = Window().partitionBy().orderBy(col("Date"))
df = df.select("*", lag("at").over(w).alias("prev_at"))

df = df.withColumn("dat", F.when(F.isnull(df.at - df.prev_at), 0)
                              .otherwise(df.at - df.prev_at))
df = df.select('Date', 'Station Name', 'sid', 'swe', 'dswe', 'sd', 'dsd', 'at', 'dat')

df.show(n = 20)

#map(lambda word: (word, 1))

#make compass rose
cr = df.select('Date', 'Station Name', 'sid')

df.select('at').rdd.map(lambda x: lit(1).toDF().show())
#cr = cr.withColumn('ww_bt', df.select('at').map(lambda x: 0))
#cr = cr.withColumn('nw_bt',
#cr = cr.withColumn('nn_bt', 
#cr = cr.withColumn('ne_bt',
#cr = cr.withColumn('ee_bt', 
#cr = cr.withColumn('se_bt', 
#cr = cr.withColumn('ss_bt',
#cr = cr.withColumn('sw_bt', 
#cr = cr.withColumn('ww_nt', 
#cr = cr.withColumn('nw_nt', 
#cr = cr.withColumn('nn_nt', 
#cr = cr.withColumn('ne_nt',
#cr = cr.withColumn('ee_nt', 
#cr = cr.withColumn('se_nt', 
#cr = cr.withColumn('ss_nt', 
#cr = cr.withColumn('sw_nt', 
#cr = cr.withColumn('ww_at', 
#cr = cr.withColumn('nw_at', 
#cr = cr.withColumn('nn_at',
#cr = cr.withColumn('ne_at',
#cr = cr.withColumn('ee_at', 
#cr = cr.withColumn('se_at', 
#cr = cr.withColumn('ss_at', 
#cr = cr.withColumn('sw_at',

cr.show(n=20)

spark.stop()
